# AcousticBrainz High-Level Sample Data Processing for Music Recommendation System

## Goals
- [x] Iterate through all track files in the high-level sample
- [x] Extract MusicBrainz ID, metadata, audio features to a dict
- [ ] Map the dict to SQL (we'll add Django ORM later)

> The code assumes you have downloaded the AcousticBrainz DB dumps in the same directory, under `acousticbrainz-highlevel-sample-json-20220623-0/highlevel/`. They can be downloaded from here: https://acousticbrainz.org/download

In [46]:
import os
import json
import sqlite3
import time
from pprint import pprint

In [47]:
def extract_data_from_json(filepath):
    """
    Returns a dict with values for corresponding audio features from the AcousticBrainz dataset.
    """
    with open(filepath, 'r') as f:
        try:
            data = json.load(f)
        except json.JSONDecodeError:
            print("Bad JSON:", filepath)
            return None

        highlevel = data.get('highlevel') or {}
        metadata = data.get('metadata') or {}
        tags = metadata.get('tags') or {}

        return {
            # metadata
            "artist":tags.get('artist', [None])[0],
            "album":tags.get('album', [None])[0],
            "title":tags.get('title', [None])[0],
            "release_date":tags.get('originaldate', [None])[0],
            "duration": metadata.get('audio_properties', {}).get('length', None),

            # high level features
            "genre":  highlevel.get('genre_dortmund', {}).get('value', None),
            "danceability":  highlevel.get('danceability', {}).get('all', {}).get('danceable', None),
            "aggressiveness": highlevel.get('mood_aggressive', {}).get('all', {}).get('aggressive', None),
            "happiness": highlevel.get('mood_happy', {}).get('all', {}).get('happy', None),
            "sadness": highlevel.get('mood_sad', {}).get('all', {}).get('sad', None),
            "relaxedness": highlevel.get('mood_relaxed', {}).get('all', {}).get('relaxed', None),
            "partyness": highlevel.get('mood_party', {}).get('all', {}).get('party', None),
            "acousticness": highlevel.get('mood_acoustic', {}).get('all', {}).get('acoustic', None),
            "electronicness": highlevel.get('mood_electronic', {}).get('all', {}).get('electronic', None),
            "instrumentalness": highlevel.get('voice_instrumental', {}).get('all', {}).get('instrumental', None),
            "tonality": highlevel.get('tonal_atonal', {}).get('all', {}).get('tonal', None),
            "brightness": highlevel.get('timbre', {}).get('all', {}).get('bright', None),
        }

In [48]:
highlevel_path = 'acousticbrainz-highlevel-sample-json-20220623/highlevel/'


# test = extract_data_from_json(os.path.join(highlevel_path, '00', '0', '000a9db8-949f-4fa2-9f40-856127df0dbc-0.json'))
# pprint(test)

json_paths = []

# walks through a branch of the directory tree, it will look at all subfolders and files recursively
for root, dirs, files in os.walk(highlevel_path):
    for name in files:
        json_paths.append(os.path.join(root, name))

db_path = "../../db.sqlite3"
print("Connecting to DB at: ", os.path.abspath(db_path))
db_conn = sqlite3.connect(db_path)
db_cursor = db_conn.cursor()

# WARNING! For development only disable safety features to increase speed
# db_cursor.execute("PRAGMA journal_mode = OFF")
# db_cursor.execute("PRAGMA synchronous = OFF")

try:
    db_cursor.execute("DROP TABLE IF EXISTS track")
    db_cursor.execute("""            
    CREATE TABLE IF NOT EXISTS track (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        artist TEXT,
        album TEXT,
        title TEXT,
        release_date DATE,
        duration REAL,
        genre TEXT,
        danceability REAL,
        aggressiveness REAL,
        happiness REAL,
        sadness REAL,
        relaxedness REAL,
        partyness REAL,
        acousticness REAL,
        electronicness REAL,
        instrumentalness REAL,
        tonality REAL,
        brightness REAL
    )
    """)
except Exception as e:
    print("Error:", e)


Connecting to DB at:  c:\src\CM3070-FP-Music-Recommendation\backend\db.sqlite3


In [49]:
records = []
start = time.time()

print(f"Will load {len(json_paths)} records")

for json_path in json_paths:
    features = extract_data_from_json(json_path)

    if features is None:
        continue

    records.append((
        features.get('artist'),
        features.get('album'),
        features.get('title'),
        features.get('release_date'),
        features.get('duration'),
        features.get('genre'),
        features.get('danceability'),
        features.get('aggressiveness'),
        features.get('happiness'),
        features.get('sadness'),
        features.get('relaxedness'),
        features.get('partyness'),
        features.get('acousticness'),
        features.get('electronicness'),
        features.get('instrumentalness'),
        features.get('tonality'),
        features.get('brightness'),
    ))

end = time.time()
print(f"Finished loading records into memory in {end - start:.2f}s, now running the SQL inserts.")

start = time.time()
db_cursor.executemany("""
    INSERT INTO track (
        artist, album, title, release_date, duration, genre, danceability,
        aggressiveness, happiness, sadness, relaxedness, partyness,
        acousticness, electronicness, instrumentalness, tonality, brightness
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""", records)

db_conn.commit()
end = time.time()
print(f"Inserted {len(records)} records in {end - start:.2f} seconds")

db_conn.close()

print("DONE")

Will load 100000 records
Finished loading records into memory in 31.82s, now running the SQL inserts.
Inserted 100000 records in 0.36 seconds
DONE
